The conda environment required for running this notebook can be installed and activated by running the following on the command line from within this folder: 
<code>conda env create -f evolution_env_gpu_only.yml </code>  \
<code>conda activate seq</code>


In [ ]:
import random
from deap import creator, base, tools, algorithms
import numpy as np
import copy
from tqdm import tqdm
import tensorflow as tf, sys, numpy as np, h5py, pandas as pd
import copy
from os.path import splitext,exists,dirname,join,basename


#mapper = {'A':[1,0,0,0],'C':[0,1,0,0],'G':[0,0,1,0],'T':[0,0,0,1],'N':[0,0,0,0]}
#worddim = len(mapper['A'])
A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

mapper = {'A':A_onehot,'C':C_onehot,'G':G_onehot,'T':T_onehot,'N':N_onehot}
worddim = len(mapper['A'])


# Function to embed sequences and another one to get reverse complements
def seq2feature(data,mapper,worddim):

    
    transformed = np.zeros([data.shape[0],1,len(data[0]),4] , dtype=np.bool )
    for i in range(data.shape[0]) :
        for j,k in enumerate(data[i]):
            #print j,k
            transformed[i,0,j] = mapper[k] 
            #print mapper[k]
    return transformed
    
def get_rc(A):
    A_r = np.flip(A,2)
    A = A_r
    A_onehot = np.array([1,0,0,0] ,  dtype=np.bool)
    C_onehot = np.array([0,1,0,0] ,  dtype=np.bool)
    G_onehot = np.array([0,0,1,0] ,  dtype=np.bool)
    T_onehot = np.array([0,0,0,1] ,  dtype=np.bool)
    N_onehot = np.array([0,0,0,0] ,  dtype=np.bool)

    for i in range(A.shape[0]) : 
        for j in range(A.shape[2]) : 
            if np.array_equal(A[i][0][j] , A_onehot) :
                A[i][0][j] = T_onehot
            elif np.array_equal(A[i][0][j] , C_onehot) :
                A[i][0][j] = G_onehot
            elif np.array_equal(A[i][0][j] , G_onehot) :
                A[i][0][j] = C_onehot
            elif np.array_equal(A[i][0][j] , T_onehot) :
                A[i][0][j] = A_onehot
            elif np.array_equal(A[i][0][j] , N_onehot) :
                A[i][0][j] = N_onehot

                
    return A



def fitness(population):

    population_nn = [];
    population_fits = [];
    for ind in range(len(population)) :
        individual_nn = ''.join(['T','G','C','A','T','T','T','T','T','T','T','C','A','C','A','T','C'] + population[ind] + ['G','G','T','T','A','C','G','G','C','T','G','T','T'] )
        population_nn.append(individual_nn)
        #population_fits.append((population[ind].count('A'),) )
    #return population_fits


    with tf.device('/cpu:0'):
        #sequences = list()
        #for individual in range(len(population)) :
        #    sequence = (''.join(population[individual]))
        #    sequences.append(sequence)
        #print sequences
        _best_model_file = join('motif_disc','best_model.ckpt')
        _best_model_file_hyper = join('motif_disc', 'hyper_search', 'best_model.ckpt')
    
        #sequences = ['NNNTGCATTTTTTTCACATCGGAATGGTGTTGTCGACCTTGCCTTCATAGTTACTTGATGTTAATGATTGGACATGTTCATCAAACCGGGTCACAGAGGTTACGGCTGTT',  'NTGCATTTTTTTCACATCTCGGCGCTTGTGGGTCGTAGCGCATATTCTGTCTACATGTACCTCTTTAATGGCTAATCACGAGGAACGCGCGAGGTAGGGTTACGGCTGTT']
        _teX = seq2feature(np.asarray(population_nn),mapper,worddim)
        _teX_rc = get_rc(list(_teX)) 
        _teY = np.zeros([np.shape(_teX)[0] , 1] , dtype=float)
        #print X, Y , X_rc , X
        sess = tf.Session()
        init=tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        #init = tf.global_variables_initializer()
        sess.run(init)

        new_saver = tf.train.import_meta_graph('%s.meta'%_best_model_file)
        new_saver.restore(sess, tf.train.latest_checkpoint('motif_disc'))
        graph = tf.get_default_graph()



        X = graph.get_tensor_by_name('Placeholder:0')
        X_rc = graph.get_tensor_by_name('Placeholder_1:0')
        Y = graph.get_tensor_by_name('Placeholder_2:0')

        model_cost = graph.get_tensor_by_name('out/model_cost:0')
        model_output = graph.get_tensor_by_name('out/model_output:0')
        #model_accuracy = graph.get_tensor_by_name('out/model_accuracy:0')

        #teX_output, teX_cost, teX_accuracy = sess.run([model_output , model_cost, model_accuracy ], feed_dict={X: _teX, X_rc: _teX_rc ,Y: _teY})

        teX_output = sess.run([model_output], feed_dict={X: _teX, X_rc: _teX_rc  ,Y: _teY})
        tuple_output = list(tuple((x),) for x in teX_output[0])
        
        return tuple_output


#fits = fitness(offspring)

In [ ]:
import random
from deap import creator, base, tools, algorithms
import numpy as np
import copy
from tqdm import tqdm
import tensorflow as tf, sys, numpy as np, h5py, pandas as pd
import copy
from os.path import splitext,exists,dirname,join,basename




args  = {'sequence_length' : 80 , 'nucleotide_frequency' :[0.25,0.25,0.25,0.25] } 
randomizer=np.random

def random_sequence_generator(randomizer,args) :
    return randomizer.choice(list('ACGT') , p=args['nucleotide_frequency'] ) 


def mutation(individual, indpb):

    for i in xrange(len(individual)):
        if random.random() < indpb:
            if individual[i]=='A' :
                individual[i] = (randomizer.choice(list('CGT') , p=[args['nucleotide_frequency'][1]/(1-args['nucleotide_frequency'][0]) ,args['nucleotide_frequency'][2]/(1-args['nucleotide_frequency'][0]) ,args['nucleotide_frequency'][3]/(1-args['nucleotide_frequency'][0]) ] ) )
            elif individual[i]=='C' :
                individual[i] = (randomizer.choice(list('AGT') , p=[args['nucleotide_frequency'][0]/(1-args['nucleotide_frequency'][1]) ,args['nucleotide_frequency'][2]/(1-args['nucleotide_frequency'][1]) ,args['nucleotide_frequency'][3]/(1-args['nucleotide_frequency'][1]) ] ) )
            elif individual[i]=='G' :
                individual[i] = (randomizer.choice(list('CGT') , p=[args['nucleotide_frequency'][2]/(1-args['nucleotide_frequency'][2]) ,args['nucleotide_frequency'][1]/(1-args['nucleotide_frequency'][2]) ,args['nucleotide_frequency'][3]/(1-args['nucleotide_frequency'][2]) ] ) )
            elif individual[i]=='T' : 
                individual[i] = (randomizer.choice(list('CGT') , p=[args['nucleotide_frequency'][0]/(1-args['nucleotide_frequency'][3]) ,args['nucleotide_frequency'][1]/(1-args['nucleotide_frequency'][3]) ,args['nucleotide_frequency'][2]/(1-args['nucleotide_frequency'][3]) ] ) )
    return individual,

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list , fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("base", random_sequence_generator , randomizer , args)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.base, n=args['sequence_length'])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutation, indpb=0.025)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=100000)

NGEN=10

for gen in tqdm(range(NGEN)):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.1, mutpb=0.1)
    fits = toolbox.evaluate(offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
    
top10 = tools.selBest(population, k=10)

print top10